## Подготовка

In [61]:
import pandas as pd 

import nltk 
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem.wordnet import WordNetLemmatizer 
import re 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import f1_score 
from sklearn.ensemble import RandomForestClassifier 
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings("ignore") 

In [63]:
# Install spaCy
import sys
!{sys.executable} -m pip install spacy



In [10]:
# Download spaCy's  'en' Model
!{sys.executable} -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.9 MB 1.3 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [64]:
import spacy

In [65]:
RANDOM_STATE = 42 

In [66]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [67]:
data.head() 

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Рассмотрим полученные данные 

In [68]:
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [71]:
len(data.query('toxic == 0')) 

143106

Лемматизируем текст 

In [70]:
nlp = spacy.load("en_core_web_sm") 

def lemmatize_text(text):
    text = text.lower()
    doc = nlp(text)
    lemm_text = " ".join([token.lemma_ for token in doc])
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
    return " ".join(cleared_text.split())

data['lemm_text'] = data['text'].apply(lemmatize_text)

data = data.drop(['text'], axis=1)

In [72]:
data.head() 

,Unnamed: 0,toxic,lemm_text
0,0,0,explanation why the edit make under my usernam...
1,1,0,d aww he match this background colour I be see...
2,2,0,hey man I be really not try to edit war it be ...
3,3,0,more I can not make any real suggestion on imp...
4,4,0,you sir be my hero any chance you remember wha...


In [73]:
nltk.download('stopwords') 
stop_words = set(nltk_stopwords.words('english')) 

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [74]:
X = data.drop(['toxic'], axis=1) 
y = data['toxic'] 

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True,stratify=y)


Применим модель TfidfVectorizer и обучим её на наших тренировочных данных 


In [76]:
count_tf_idf = TfidfVectorizer(stop_words=stop_words,analyzer='word') 
tf_idf = count_tf_idf.fit_transform(X_train['lemm_text']) 
tf_idf.shape

(143362, 144162)

Трансформируем тестовую выборку 

In [77]:
tf_test = count_tf_idf.transform(X_test['lemm_text'])
tf_test.shape

(15930, 144162)

Была проведена подготовка полученных данных к обучению моделей. Мы лемматизировали данные, избавились от стоп-слов и разделили на тренировочную и тестовую выборки. 

## Обучение

Обучим модель DecisionTreeClassifier 

In [46]:
model_dt = DecisionTreeClassifier() 
hyperparams = [{'max_depth':[x for x in range(50,100,2)],
                'random_state':[42]}]

In [47]:
grid_dt = GridSearchCV(model_dt, hyperparams, scoring='f1',cv=3) 
grid_dt.fit(tf_idf, y_train) 
grid_dt.best_params_ 

{'max_depth': 96, 'random_state': 42}

In [48]:
grid_dt.best_score_ 

0.7243812487727214

Метрика f1 показывает недостаточную точность предсказаний 0,72, по условия значение должно быть не менее 0,75 
Попробуем модель RandomForestClassifier 

In [78]:
model_rf = RandomForestClassifier(random_state=RANDOM_STATE) 

In [79]:
hyperparams = [{'max_depth':[x for x in range(3, 15)], 
                'random_state':[42], 
               'class_weight':['balanced', None]}] 
grid_rf = GridSearchCV(model_rf, hyperparams, scoring='f1',cv=3) 
grid_rf.fit(tf_idf, y_train) 
grid_rf.best_params_ 

{'class_weight': 'balanced', 'max_depth': 14, 'random_state': 42}

In [80]:
grid_rf.best_score_ 

0.3774784813385657

Снова недостаточноная точность 
Обучим LogisticRegression и подберём параметры при помощи GridSearchCV 

In [34]:
clf_lr = LogisticRegression(random_state=12345, solver='sag',class_weight='balanced')
parametrs_lr = { 'C': range (1, 10),
              'max_iter': range (10,15,2),
              }
grid_lr = GridSearchCV(clf_lr, parametrs_lr, cv=3, scoring='f1')
grid_lr.fit(tf_idf, y_train)
grid_lr.best_params_

{'C': 4, 'max_iter': 14}

In [35]:
grid_lr.best_score_

0.760801612341031

In [82]:
comparison_tabl = pd.DataFrame(index=['F1-мера'], columns=['Дерево решений','Cлучайный лес','Логистическая регрессия'])
comparison_tabl['Дерево решений'] = grid_dt.best_params_.round(2)
comparison_tabl['Cлучайный лес'] = grid_rf.best_score_.round(2) 
comparison_tabl['Логистическая регрессия'] = grid_lr.best_score_.round(2) 
comparison_tabl.T

,F1-мера
Дерево решений,NaN
Cлучайный лес,0.38
Логистическая регрессия,0.76


Лучшее значение f1 показала Логистическая регрессия, проверим ее на тестовой выборке 

In [83]:
predicted_lr= grid_lr.predict(tf_test) 
f1_lr_test = f1_score(y_test, predicted_lr) 
f1_lr_test 

0.7734837799717914

Качество предсказаний отвечает условиям задачи, f1 не менее 0,75 

Были обучены модели DecisionTreeClassifier, RandomForestClassifier и LogisticRegression. Первые две модели не отвечают условиям значения метркии качества, показав f1 менее 0,75. 
Модель LogisticRegression показала лучший результат, значение метрики f1 = 0,77 при параметрах {'C': 4, 'max_iter': 12} 


## Выводы

Для интернет-магазина «Викишоп» была реализована задача найти инструмент, который ищет токсичные комментарии и отправляет их на модерацию. При этом метрика результата предсказания наилучшей модели по мерке F1 должна быть не меньше 0.75. 
После загрузки полученных данных - комментариев была проведена подготовка их к обучения. 
Затем были обучены три модели - DecisionTreeClassifier, LogisticRegression и RandomForestClassifier. По итогам сравнения лучший результат показала модель LogisticRegression с параметрами С = 4 и max_iter = 12, она набрала значение метрики F1 0,76. 